In [1]:
!pip install transformers torch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("/kaggle/input/span-and-text/kannada_final (1)training dataset.tsv", sep="\",")

In [3]:
df

,spans,text
0,"""[0, 1, 2, 3, 4, 5, 6, 7]",Ade old same story
1,"""[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, ...",ಬ್ರೋ ಅವರು ಗೆ ದೇಶದ ಬಗ್ಗೆ ಅಭಿಮಾನ ಇಲ್ಲಾ ಬಿಡಿ
2,"""[57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, ...",ದೇಶಧ್ರೋಹಿಗಳು ಡಿಸ್ ಲೈಕ್ ಮಾಡಿದರೆ ನೀವು ನಿಜವಾದ ದೇಶ...
3,"""[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]",Bindu gowda ge super agi ugididdeera
4,"""[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]",Kannadadalli mado badlu hindinalli madidre sup...
...,...,...
1795,"""[55, 56, 57, 58, 59, 60, 61, 62]",Ivarukkku eppodhum thalaivar kalaigner lightaa...
1796,"""[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, ...",Trailer Nala irukanu oru than comment pandranu...
1797,"""[24, 25, 26, 27, 28, 29, 102, 103, 104, 105, ...",Wigpathy Visay na Padam Flop than ithula Kabal...
1798,"""[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]",Vikram ella styleum set aaguthu.. Namba moonji...


In [4]:
def rem(text):
    return(text[1:])
df['spans'] = df['spans'].apply(rem)

In [5]:
texts = df['text'].tolist()
spans = [eval(span) for span in df['spans'].tolist()]
spans[1]
df

,spans,text
0,"[0, 1, 2, 3, 4, 5, 6, 7]",Ade old same story
1,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",ಬ್ರೋ ಅವರು ಗೆ ದೇಶದ ಬಗ್ಗೆ ಅಭಿಮಾನ ಇಲ್ಲಾ ಬಿಡಿ
2,"[57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 6...",ದೇಶಧ್ರೋಹಿಗಳು ಡಿಸ್ ಲೈಕ್ ಮಾಡಿದರೆ ನೀವು ನಿಜವಾದ ದೇಶ...
3,"[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]",Bindu gowda ge super agi ugididdeera
4,"[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]",Kannadadalli mado badlu hindinalli madidre sup...
...,...,...
1795,"[55, 56, 57, 58, 59, 60, 61, 62]",Ivarukkku eppodhum thalaivar kalaigner lightaa...
1796,"[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 9...",Trailer Nala irukanu oru than comment pandranu...
1797,"[24, 25, 26, 27, 28, 29, 102, 103, 104, 105, 1...",Wigpathy Visay na Padam Flop than ithula Kabal...
1798,"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]",Vikram ella styleum set aaguthu.. Namba moonji...


In [6]:
!pip install googletrans==4.0.0-rc1
!pip install langdetect

  Preparing metadata (setup.py) ... - \ | done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=4ee9a6e8f3c3ca56abae52b9616b727771660f3742746a22478528d9c8d5ad5c
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalli

In [7]:
from googletrans import Translator, LANGUAGES
from langdetect import detect

translator = Translator()

def translate_or_transliterate(text):
    try:
        # Detect the language of the text
        detected_lang = detect(text)

        # If the text is in Kannada, translate it to English
        if detected_lang == 'kn':
            return translator.translate(text, src='kn', dest='en').text
        
#         # If the text is Kannada written in Roman script, transliterate (this is a placeholder, actual transliteration might need a different approach)
#         if detected_lang == 'en' and 'some condition to identify Romanized Kannada':
#             # Transliterate here (this is just a placeholder)
#             return 'Transliterated Text'

        # If the text is already in English, return as is
#         if detected_lang == 'en':
#             return text
        else:
            return text

    except Exception as e:
        return text

df['translated_text'] = df['text'].apply(translate_or_transliterate)
df


,spans,text,translated_text
0,"[0, 1, 2, 3, 4, 5, 6, 7]",Ade old same story,Ade old same story
1,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",ಬ್ರೋ ಅವರು ಗೆ ದೇಶದ ಬಗ್ಗೆ ಅಭಿಮಾನ ಇಲ್ಲಾ ಬಿಡಿ,Bro let him be good about the country
2,"[57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 6...",ದೇಶಧ್ರೋಹಿಗಳು ಡಿಸ್ ಲೈಕ್ ಮಾಡಿದರೆ ನೀವು ನಿಜವಾದ ದೇಶ...,"If the patriots discriminate, you should be sh..."
3,"[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]",Bindu gowda ge super agi ugididdeera,Bindu gowda ge super agi ugididdeera
4,"[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]",Kannadadalli mado badlu hindinalli madidre sup...,Kannadadalli mado badlu hindinalli madidre sup...
...,...,...,...
1795,"[55, 56, 57, 58, 59, 60, 61, 62]",Ivarukkku eppodhum thalaivar kalaigner lightaa...,Ivarukkku eppodhum thalaivar kalaigner lightaa...
1796,"[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 9...",Trailer Nala irukanu oru than comment pandranu...,Trailer Nala irukanu oru than comment pandranu...
1797,"[24, 25, 26, 27, 28, 29, 102, 103, 104, 105, 1...",Wigpathy Visay na Padam Flop than ithula Kabal...,Wigpathy Visay na Padam Flop than ithula Kabal...
1798,"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]",Vikram ella styleum set aaguthu.. Namba moonji...,Vikram ella styleum set aaguthu.. Namba moonji...


In [8]:
df['text'] = df['translated_text']

In [9]:
import ast
from nltk.tokenize import word_tokenize

def convert_spans_to_bio(text, char_spans_str):
    # Convert string representation of list to actual list
    char_spans = ast.literal_eval(char_spans_str)

    tokens = word_tokenize(text)
    bio_tags = ['O'] * len(tokens)
    token_spans = []
    start = 0

    for token in tokens:
        token_spans.append((start, start + len(token)))
        start += len(token) + 1  # +1 for the space

    for start_char, end_char in zip(char_spans[::2], char_spans[1::2]):
        for i, (start_token, end_token) in enumerate(token_spans):
            if start_token < end_char and end_token > start_char:
                bio_tags[i] = 'B' if start_token == start_char else 'I'

    return bio_tags

# Apply the conversion to the DataFrame
df['bio_tags'] = df.apply(lambda row: convert_spans_to_bio(row['text'], row['spans']), axis=1)


In [10]:
df

,spans,text,translated_text,bio_tags
0,"[0, 1, 2, 3, 4, 5, 6, 7]",Ade old same story,Ade old same story,"[I, I, O, O]"
1,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",Bro let him be good about the country,Bro let him be good about the country,"[O, O, O, O, O, I, I, I]"
2,"[57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 6...","If the patriots discriminate, you should be sh...","If the patriots discriminate, you should be sh...","[O, O, O, O, O, O, O, O, O, O, O, I, O, I, I, ..."
3,"[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]",Bindu gowda ge super agi ugididdeera,Bindu gowda ge super agi ugididdeera,"[O, O, O, O, O, I]"
4,"[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]",Kannadadalli mado badlu hindinalli madidre sup...,Kannadadalli mado badlu hindinalli madidre sup...,"[O, I, I, O, O, O, O]"
...,...,...,...,...
1795,"[55, 56, 57, 58, 59, 60, 61, 62]",Ivarukkku eppodhum thalaivar kalaigner lightaa...,Ivarukkku eppodhum thalaivar kalaigner lightaa...,"[O, O, O, O, O, O, I]"
1796,"[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 9...",Trailer Nala irukanu oru than comment pandranu...,Trailer Nala irukanu oru than comment pandranu...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, I, I, ..."
1797,"[24, 25, 26, 27, 28, 29, 102, 103, 104, 105, 1...",Wigpathy Visay na Padam Flop than ithula Kabal...,Wigpathy Visay na Padam Flop than ithula Kabal...,"[O, O, O, O, I, O, O, O, O, O, O, O, O, O, O, ..."
1798,"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]",Vikram ella styleum set aaguthu.. Namba moonji...,Vikram ella styleum set aaguthu.. Namba moonji...,"[O, O, O, O, O, O, O, O, O, O, I, I]"


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Parameters
vocab_size = 5000  # Adjust as needed
max_len = 128  # Max length of sequences
embedding_dim = 100  # Embedding dimensions

# Initialize tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token='UNK')
tokenizer.fit_on_texts(df['text'])

# Tokenize and pad sequences
sequences = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# Convert BIO tags to numerical format and pad
tag2idx = {'O': 0, 'B': 1, 'I': 2}
y = [[tag2idx.get(tag) for tag in bio_tags] for bio_tags in df['bio_tags']]
y = pad_sequences(y, maxlen=max_len, padding='post', value=tag2idx['O'])
y = np.array([np.eye(len(tag2idx))[i] for i in y])  # One-hot encoding

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [12]:
!pip install tf2crf

In [13]:
df

,spans,text,translated_text,bio_tags
0,"[0, 1, 2, 3, 4, 5, 6, 7]",Ade old same story,Ade old same story,"[I, I, O, O]"
1,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",Bro let him be good about the country,Bro let him be good about the country,"[O, O, O, O, O, I, I, I]"
2,"[57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 6...","If the patriots discriminate, you should be sh...","If the patriots discriminate, you should be sh...","[O, O, O, O, O, O, O, O, O, O, O, I, O, I, I, ..."
3,"[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]",Bindu gowda ge super agi ugididdeera,Bindu gowda ge super agi ugididdeera,"[O, O, O, O, O, I]"
4,"[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]",Kannadadalli mado badlu hindinalli madidre sup...,Kannadadalli mado badlu hindinalli madidre sup...,"[O, I, I, O, O, O, O]"
...,...,...,...,...
1795,"[55, 56, 57, 58, 59, 60, 61, 62]",Ivarukkku eppodhum thalaivar kalaigner lightaa...,Ivarukkku eppodhum thalaivar kalaigner lightaa...,"[O, O, O, O, O, O, I]"
1796,"[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 9...",Trailer Nala irukanu oru than comment pandranu...,Trailer Nala irukanu oru than comment pandranu...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, I, I, ..."
1797,"[24, 25, 26, 27, 28, 29, 102, 103, 104, 105, 1...",Wigpathy Visay na Padam Flop than ithula Kabal...,Wigpathy Visay na Padam Flop than ithula Kabal...,"[O, O, O, O, I, O, O, O, O, O, O, O, O, O, O, ..."
1798,"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]",Vikram ella styleum set aaguthu.. Namba moonji...,Vikram ella styleum set aaguthu.. Namba moonji...,"[O, O, O, O, O, O, O, O, O, O, I, I]"


In [14]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.models import Model

# Model Architecture without CRF
input = Input(shape=(max_len,))
model = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=128, return_sequences=True))(model)
out = TimeDistributed(Dense(len(tag2idx), activation="softmax"))(model)  # Output layer

model = Model(input, out)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding (Embedding)       (None, 128, 100)          500000    
                                                                 
 dropout (Dropout)           (None, 128, 100)          0         
                                                                 
 bidirectional (Bidirection  (None, 128, 256)          234496    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 128, 3)            771       
 ributed)                                                        
                                                                 
Total params: 735267 (2.80 MB)
Trainable params: 735267 (2.80

In [15]:
history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
51/51 [==============================] - 17s 205ms/step - loss: 0.2144 - accuracy: 0.9418 - val_loss: 0.0807 - val_accuracy: 0.9636
Epoch 2/5
51/51 [==============================] - 7s 143ms/step - loss: 0.0763 - accuracy: 0.9636 - val_loss: 0.0786 - val_accuracy: 0.9644
Epoch 3/5
51/51 [==============================] - 5s 104ms/step - loss: 0.0705 - accuracy: 0.9673 - val_loss: 0.0794 - val_accuracy: 0.9622
Epoch 4/5
51/51 [==============================] - 5s 92ms/step - loss: 0.0651 - accuracy: 0.9720 - val_loss: 0.0782 - val_accuracy: 0.9641
Epoch 5/5
51/51 [==============================] - 2s 42ms/step - loss: 0.0594 - accuracy: 0.9759 - val_loss: 0.0779 - val_accuracy: 0.9648


In [16]:
new_df = pd.read_csv("/kaggle/input/span-and-text/kannada_test_EACL24 (1).csv",header=None,names=['text'])

In [17]:
new_df

,text
0,#rashmikamandhana bad girl
1,#ಇದು ಚರಿತ್ರೆ ಸ್ರೃಷ್ಟಿಸುವ ಅವತಾರ nBaground music 🤺🤘
2,@#win 12 nodidini sir
3,@A.R.W tumbad tanhaji andhadhun aise bahot h...
4,@Ajaya shetty sari bidi
...,...
439,Yes bro avaga song ajneesh sir voice alli ittu
440,Yes hange kick kodta hogutte.. ಒಳ್ಳೆ synch ಮಾಡ...
441,Yes song super
442,You have bright future


In [18]:
# Convert spans to BIO tags for the new dataset
# new_df['bio_tags'] = new_df.apply(lambda row: convert_spans_to_bio(row['text'], row['spans']), axis=1)

# Tokenize and pad new dataset
new_sequences = tokenizer.texts_to_sequences(new_df['text'])
X_new = pad_sequences(new_sequences, maxlen=max_len, padding='post', truncating='post')

# Predict
predictions = model.predict(X_new)

predictions[0]


14/14 [==============================] - 1s 7ms/step


array([[4.86551136e-01, 2.07148753e-02, 4.92734045e-01],
       [4.86104369e-01, 2.09144540e-02, 4.92981106e-01],
       [6.97252929e-01, 1.80555321e-02, 2.84691572e-01],
       [9.01204705e-01, 1.00619039e-02, 8.87334347e-02],
       [9.62265491e-01, 4.94811684e-03, 3.27863060e-02],
       [9.88090158e-01, 2.02982710e-03, 9.87993088e-03],
       [9.96515155e-01, 7.59714399e-04, 2.72514066e-03],
       [9.98918533e-01, 2.89466203e-04, 7.91976403e-04],
       [9.99604523e-01, 1.23272082e-04, 2.72280391e-04],
       [9.99822676e-01, 6.13195152e-05, 1.16123214e-04],
       [9.99904037e-01, 3.54748154e-05, 6.04764537e-05],
       [9.99939919e-01, 2.32211660e-05, 3.68531801e-05],
       [9.99958158e-01, 1.66953487e-05, 2.52005721e-05],
       [9.99968410e-01, 1.28729689e-05, 1.87328642e-05],
       [9.99974728e-01, 1.04599685e-05, 1.48059844e-05],
       [9.99978900e-01, 8.84466226e-06, 1.22534748e-05],
       [9.99981761e-01, 7.71201576e-06, 1.05047675e-05],
       [9.99983788e-01, 6.88778

In [19]:
max_value_indices = np.argmax(predictions[1], axis=1)
max_value_indices

array([0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
res = []
for i in range(len(predictions)):    
    x = np.argmax(predictions[i], axis=1)
    y = np.where(x == 2)[0]
    res.append(list(y))
res
new_df['final_span']=res

In [21]:
new_df

,text,final_span
0,#rashmikamandhana bad girl,"[0, 1]"
1,#ಇದು ಚರಿತ್ರೆ ಸ್ರೃಷ್ಟಿಸುವ ಅವತಾರ nBaground music 🤺🤘,"[3, 4, 5, 6]"
2,@#win 12 nodidini sir,[]
3,@A.R.W tumbad tanhaji andhadhun aise bahot h...,"[33, 34]"
4,@Ajaya shetty sari bidi,[]
...,...,...
439,Yes bro avaga song ajneesh sir voice alli ittu,[]
440,Yes hange kick kodta hogutte.. ಒಳ್ಳೆ synch ಮಾಡ...,[]
441,Yes song super,[]
442,You have bright future,"[0, 2]"


In [22]:
new_df.to_csv("result.csv")

In [23]:
new_df['final_span'].value_counts()

final_span
[]                                                                                 214
[1]                                                                                 25
[1, 2]                                                                              13
[0, 1]                                                                              10
[2]                                                                                  9
                                                                                  ... 
[2, 4]                                                                               1
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]      1
[10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]                         1
[4, 5, 6, 7, 8, 10]                                                                  1
[0, 2]                                                                               1
Name: count, Length: 108, dtype:

In [24]:
# # Assuming predictions is the output from model.predict

# # Function to convert numerical indices back to BIO tags
# def indices_to_tags(predictions, idx2tag):
#     tag_sequences = []
#     for sequence in predictions:
#         tag_sequence = [idx2tag[np.argmax(tag)] for tag in sequence]
#         tag_sequences.append(tag_sequence)
#     return tag_sequences

# # Create an inverse mapping from numerical index to tag
# idx2tag = {v: k for k, v in tag2idx.items()}

# # Convert predictions to BIO tags
# bio_tag_predictions = indices_to_tags(predictions, idx2tag)
# bio_tag_predictions[0]

In [25]:
# # new_df = pd.read_csv("/kaggle/input/span-and-text/kannada_test_EACL24 (1).csv",header=None, names=['text'] )
# new_df = df
# new_sequences = tokenizer.texts_to_sequences(new_df['translated_text'])
# new_X = pad_sequences(new_sequences, maxlen=max_len, padding='post', truncating='post')

In [26]:
# new_df = df

In [27]:
# # Predict using the model
# predictions = model.predict(new_X)
# predicted_bio_tags = np.argmax(predictions, axis=-1)  # Convert probabilities to class labels


In [28]:
# def bio_tags_to_spans(bio_sequence, tokenizer):
#     spans = []
#     current_span = []

#     for idx, tag in enumerate(bio_sequence):
#         if tag == tag2idx['B']:
#             if current_span:
#                 spans.append(current_span)
#             current_span = [idx]
#         elif tag == tag2idx['I']:
#             if current_span:
#                 current_span.append(idx)
#         else:
#             if current_span:
#                 spans.append(current_span)
#                 current_span = []

#     # Add the last span
#     if current_span:
#         spans.append(current_span)

#     return spans

# # Apply the conversion to predicted BIO tags
# new_df['predicted_spans'] = [bio_tags_to_spans(bio_sequence, tokenizer) for bio_sequence in predicted_bio_tags]


In [29]:
# new_df['predicted_spans']

In [30]:
# # Inspecting more data from the training set
# sample_size = 100  # Increase sample size
# sample_indices = np.random.choice(range(len(X_train)), size=sample_size, replace=False)
# sample_X = X_train[sample_indices]
# sample_y_true = y_train[sample_indices]

# sample_predictions = model.predict(sample_X)
# sample_predicted_bio_tags = np.argmax(sample_predictions, axis=-1)

# empty_true_spans, empty_predicted_spans = 0, 0

# for i in range(sample_size):
#     true_spans = bio_tags_to_spans(sample_y_true[i], tag2idx)
#     predicted_spans = bio_tags_to_spans(sample_predicted_bio_tags[i], tag2idx)

#     if not true_spans:
#         empty_true_spans += 1
#     if not predicted_spans:
#         empty_predicted_spans += 1

#     if i < 10:  # Print first 10 for inspection
#         print(f"True spans: {true_spans}")
#         print(f"Predicted spans: {predicted_spans}")
#         print()

# print(f"Empty true spans in sample: {empty_true_spans}/{sample_size}")
# print(f"Empty predicted spans in sample: {empty_predicted_spans}/{sample_size}")


In [31]:
# import ast
# from transformers import AutoTokenizer, AutoModelForTokenClassification


# def convert_char_spans_to_token_labels(text, char_spans, tokenizer):
#     token_labels = ['O'] * len(text)  # Initialize all labels as 'O'
#     char_spans = ast.literal_eval(char_spans)  # Convert string representation to list

#     tokens = tokenizer.tokenize(text)
#     token_positions = tokenizer(text, return_offsets_mapping=True)["offset_mapping"]

#     for start, end in zip(char_spans[::2], char_spans[1::2]):
#         for idx, (tok_start, tok_end) in enumerate(token_positions):
#             if tok_start >= start and tok_end <= end:
#                 token_labels[idx] = 'B' if tok_start == start else 'I'
    
#     return tokens, token_labels

# # Example usage with the tokenizer from a pre-trained model
# tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# # Assuming df is your DataFrame with 'spans' and 'text' columns
# df['token_labels'] = df.apply(lambda x: convert_char_spans_to_token_labels(x['text'], x['spans'], tokenizer), axis=1)

# model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# # Define label-to-index mapping
# label2idx = {'O': 0, 'B': 1, 'I': 2}

# def align_labels_with_tokens(token_labels_list, label2idx):
#     aligned_labels_list = []
#     for token_labels in token_labels_list:
#         aligned_labels = [label2idx[label] if label in label2idx else label2idx['O'] for label in token_labels]
#         aligned_labels_list.append(aligned_labels)
#     return aligned_labels_list

# df['aligned_labels'] = df['token_labels'].apply(lambda x: align_labels_with_tokens(x, label2idx))




In [32]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import AdamW

# # Assuming df is your DataFrame with 'text' and 'aligned_labels'

# class NERDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_len):
#         self.texts = texts
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         labels = self.labels[idx]

#         encoding = self.tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             return_token_type_ids=False,
#             padding='max_length',
#             truncation=True,
#             return_attention_mask=True,
#             return_tensors='pt',
#         )

#         # Adjust labels
#         label_length = len(labels)
#         adjusted_labels = labels + [0] * (self.max_len - label_length)

#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'labels': torch.tensor(adjusted_labels, dtype=torch.long)
#         }

# # Define some parameters
# max_len = 128
# batch_size = 16

# # Prepare the dataset
# dataset = NERDataset(df['text'].tolist(), df['aligned_labels'].tolist(), tokenizer, max_len)
# data_loader = DataLoader(dataset, batch_size=batch_size)
# model_name = "bert-base-multilingual-cased"  # or "bert-base-cased" for English

# # Load the model
# model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label2idx))

# # Move model to GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Optimizer
# optimizer = AdamW(model.parameters(), lr=2e-5)

# # Training loop
# model.train()
# for epoch in range(3):  # num of epochs
#     for batch in data_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss

#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         print(f"Epoch: {epoch}, Loss: {loss.item()}")


In [33]:
# texts = df['text'].tolist()
# spans = [eval(span) for span in df['spans'].tolist()]
# spans[1]

In [34]:
# i=0
# for txt in texts:
#     print(spans[i])
#     for sp in spans[i]:
#         print(txt[sp])
#     i=i+1

In [35]:
# temp_lst = []
# i=0
# for txt in texts:    
#     temp=""
# #     print(spans[i])
#     for sp in spans[i]:
#         try:
#             temp+=txt[int(sp)]
#         except:
#             continue
#     i=i+1
# #     print(temp)
#     temp_lst.append(temp)
# df['span_words']=temp_lst

In [36]:
# df

In [37]:
# new_df = df[['text','span_words']]
# new_df

In [38]:
# def bio_tagging(sentence, phrase):
#     tokens = sentence.split()
#     phrase_tokens = phrase.split()

#     tags = ['O'] * len(tokens)

#     try:
#         start_idx = tokens.index(phrase_tokens[0])

#         tags[start_idx] = 'B'
#         for i in range(1, len(phrase_tokens)):
#             if start_idx + i < len(tokens):
#                 tags[start_idx + i] = 'I'
#     except ValueError:
#         pass

#     return tags

# # Create BIO tags for each sentence
# df['bio_tags'] = df.apply(lambda row: bio_tagging(row['text'], row['span_words']), axis=1)

In [39]:
# df

In [40]:
# import torch
# from transformers import BertTokenizerFast
# from torch.utils.data import Dataset, DataLoader

# tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

# def bio_tagging(sentence, phrase):
#     tokens = sentence.split()
#     phrase_tokens = phrase.split()

#     tags = ['O'] * len(tokens)

#     try:
#         start_idx = tokens.index(phrase_tokens[0])

#         tags[start_idx] = 'B'
#         for i in range(1, len(phrase_tokens)):
#             if start_idx + i < len(tokens):
#                 tags[start_idx + i] = 'I'
#     except ValueError:
#         pass

#     return tags

# df['bio_tags'] = df.apply(lambda row: bio_tagging(row['text'], row['span_words']), axis=1)

# class SentenceDataset(Dataset):
#     def __init__(self, sentences, tags, tokenizer, max_len):
#         self.sentences = sentences
#         self.tags = tags
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         sentence = self.sentences[idx]
#         word_labels = self.tags[idx]

#         encoding = self.tokenizer(sentence.split(), is_split_into_words=True, return_offsets_mapping=True, padding='max_length', truncation=True, max_length=self.max_len)
#         labels = [label for word in word_labels for label in [word] + ['O'] * (len(self.tokenizer.tokenize(word)) - 1)]

#         # Padding
#         labels += ['O'] * (self.max_len - len(labels))

#         return {
#             'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
#             'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
#             'labels': torch.tensor([label_to_id[label] for label in labels], dtype=torch.long)  # Convert labels to numerical IDs
#         }

# label_to_id = {'O': 0, 'B': 1, 'I': 2}

# dataset = SentenceDataset(df['text'].tolist(), df['bio_tags'].tolist(), tokenizer, max_len=128)

# # Split dataset into training and validation
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# # DataLoader
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# # Model training setup (not shown here for brevity)


In [41]:
# from transformers import BertForTokenClassification, AdamW
# from torch.utils.data import DataLoader
# from sklearn.model_selection import train_test_split
# import torch

# # ... [Previous code for data preparation and DataLoader setup] ...

# # Model setup
# model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3) # Adjust num_labels based on your tagging scheme

# # Training setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # Training loop
# num_epochs = 3  # Adjust the number of epochs based on your requirement
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         optimizer.zero_grad()

#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     avg_train_loss = total_loss / len(train_loader)
#     print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}")

#     # Validation step
#     model.eval()
#     total_val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss

#             total_val_loss += loss.item()

#     avg_val_loss = total_val_loss / len(val_loader)
#     print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")

# print('Finished Training')


In [42]:
# def extract_substring(model, tokenizer, text, device):
#     # Tokenize the input text
#     inputs = tokenizer.encode_plus(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")

#     # Move tensors to the same device as the model
#     input_ids = inputs['input_ids'].to(device)
#     attention_mask = inputs['attention_mask'].to(device)

#     # Predict
#     model.eval()
#     with torch.no_grad():
#         outputs = model(input_ids, attention_mask=attention_mask)
#     logits = outputs.logits
#     # Convert logits to label IDs
#     label_ids = torch.argmax(logits, dim=2)
# #     print(label_ids)

#     # Convert IDs to labels
#     labels = [id_to_label[label_id] for label_id in label_ids[0].cpu().numpy()]

#     # Extract tokens corresponding to 'B' and 'I' tags
#     tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
#     extracted_tokens = [token for token, label in zip(tokens, labels) if label in ["B", "I"]]

#     # Convert tokens back to string
#     extracted_string = tokenizer.convert_tokens_to_string(extracted_tokens)
# #     print("ext_str",extracted_string)
#     if(extracted_string==""):
#         return "nil"
#     return extracted_string.strip()

# # Usage
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# id_to_label = {0: 'O', 1: 'B', 2: 'I'}  # Reverse mapping from ID to label

# # Example text
# text = "@Troll Stupid Fans heno helkodthidallla  adunna helkodo munche neenu kaltho henge irbeku antha suvar gandu anthidde ivga ninge baididke thika huritha"
# extracted_substring = extract_substring(model, tokenizer, text, device)
# print(extracted_substring)


In [43]:
# new_df['pred_span'] = new_df['text']

In [44]:
# new_df['extracted_substring'] = new_df['text'].apply(lambda x: extract_substring(model, tokenizer, x, device))

In [45]:
# new_df

In [46]:
# new_df['extracted_substring'].value_counts()

In [47]:
# df2=pd.read_csv("/kaggle/input/span-and-text/kannada_test_EACL24 (1).csv",header=None, names=['text'] )

In [48]:
# df2

In [49]:
# df2['extracted_substring'] = df2['text'].apply(lambda x: extract_substring(model, tokenizer, x, device))

In [50]:
# df2

In [51]:
# df2['extracted_substring'].value_counts()

In [52]:
# !pip install sklearn-crfsuite

In [53]:
# df = pd.read_csv("/kaggle/input/span-and-text/kannada_final (1).tsv", sep="\",")

In [54]:
# def rem(text):
#     return(text[1:])
# df['spans'] = df['spans'].apply(rem)
# # df

In [55]:
# import pandas as pd
# import sklearn_crfsuite
# from sklearn_crfsuite import metrics
# from sklearn.model_selection import train_test_split

# # Example DataFrame
# data = {
#     'text': ["Ade old same story", "ಬ್ರೋ ಅವರು ಗೆ ದೇಶದ ಬಗ್ಗೆ ಅಭಿಮಾನ ಇಲ್ಲಾ ಬಿಡಿ", "..."],
#     'span_words': ["Ade old", "ಅಭಿಮಾನ ಇಲ್ಲಾ ಬಿಡಿ", "..."]
# }
# df = pd.DataFrame(data)

# def bio_tagging(sentence, phrase):
#     tokens = sentence.split()
#     phrase_tokens = phrase.split()

#     tags = ['O'] * len(tokens)

#     try:
#         start_idx = tokens.index(phrase_tokens[0])

#         tags[start_idx] = 'B'
#         for i in range(1, len(phrase_tokens)):
#             if start_idx + i < len(tokens):
#                 tags[start_idx + i] = 'I'
#     except ValueError:
#         pass

#     return tags

# # Adding BIO tags to the DataFrame
# df['bio_tags'] = df.apply(lambda row: bio_tagging(row['text'], row['span_words']), axis=1)

# # Feature extraction function
# def word2features(sent, i):
#     word = sent[i]

#     features = {
#         'bias': 1.0,
#         'word.lower()': word.lower(),
#         'word[-3:]': word[-3:],
#         'word[-2:]': word[-2:],
#         'word.isupper()': word.isupper(),
#         'word.istitle()': word.istitle(),
#         'word.isdigit()': word.isdigit(),
#     }

#     if i > 0:
#         word1 = sent[i-1]
#         features.update({
#             '-1:word.lower()': word1.lower(),
#             '-1:word.istitle()': word1.istitle(),
#             '-1:word.isupper()': word1.isupper(),
#         })
#     else:
#         features['BOS'] = True

#     if i < len(sent)-1:
#         word1 = sent[i+1]
#         features.update({
#             '+1:word.lower()': word1.lower(),
#             '+1:word.istitle()': word1.istitle(),
#             '+1:word.isupper()': word1.isupper(),
#         })
#     else:
#         features['EOS'] = True

#     return features

# def prepare_data(df):
#     X, y = [], []
#     for _, row in df.iterrows():
#         tokens = row['text'].split()
#         bio_tags = row['bio_tags']

#         X.append([word2features(tokens, i) for i in range(len(tokens))])
#         y.append(bio_tags)
    
#     return X, y

# X, y = prepare_data(df)

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
